In [1]:
# 미국 텍사스 주의 여행자 수를 분석하고 싶다고 하자.
# 데이터 집합이 도시별 방문자 수 (단위: 100만 명/년)라고 가정하자. 이떄 각 도시가 전체 여행자 수 중에서 차지하는 비율을 계산

# 1년간 전체 여행자 수를 계산후 비율 구하기 -> 입력 전체의 합, 합계로 각 도시의 방문자 수를 나누는 정규화 함수 필요

def normalize(numbers):
    total = sum(numbers)
    result = []
    
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
        
    return result

visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [4]:
# 확장성을 키우기 위해 특정 경로에 방문자수를 읽어 비율을 구함 

def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)
            
it = read_visits('my_numbers.txt')
percentages = normalize(it)
print(percentages)  # why? sum(numbers) 에서 Generator를 모두 소모하기 때문에 result를 구할때 수모할 generator가 없다

[]


In [5]:
def normalize_copy(numbers):
    numbers_copy = list(numbers) # 이터레이터 명시적 복사
    total = sum(numbers_copy)
    result = []
    
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
        
    return result

it = read_visits('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages) 
# 이 방법의 문제는 메모리를 엄청나게 사용할 수 도 있다는 것이다.

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [7]:
def normalize_func(get_iter):
    total = sum(get_iter()) # 호출할때마다 iterator
    result = []
    for value in get_iter():
        percent = 100 * value / total
        result.append(percent)
        
    return result

path = 'my_numbers.txt'
percentages = normalize_func(lambda: read_visits(path)) # 작동은 됨!
print(percentages)
assert sum(percentages) == 100.0 

# 그러나 이렇게 람다함수를 parameter로 넘기는건 좋지 않아보인다

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [13]:
# 더 나은 방법 More! 
# Iterator Protocol

class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)
                
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [15]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers: # 이터레이터는 나쁨!
        raise TypeError('컨테이너를 제공해야합니다!')
    
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
        
    return result

from collections.abc import Iterator
def normalize_defensive(numbers):
    if isinstance(numbers, Iterator): # 이터레이터는 나쁨!
        raise TypeError('컨테이너를 제공해야합니다!')
    
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
        
    return result

visits = ReadVisits(path)
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0
